In [ ]:
pip install requests beautifulsoup4 pandas selenium

In [2]:
# Importing necessary libraries
import requests  # For fetching the webpage
from bs4 import BeautifulSoup  # For parsing the webpage
import pandas as pd

In [3]:
# URL of the webpage to scrape
url = 'https://starbucksmenuprices.com/'

# Send a request to the server
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    print("Successfully fetched the webpage!")
else:
    print(f"Failed to fetch webpage. Status code: {response.status_code}")

Successfully fetched the webpage!


In [5]:
# Parse the webpage content with BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Print the HTML content to understand its structure
#print(soup.prettify())

In [6]:
# Find all <ul> elements containing the country links
sections = soup.find_all('ul')  # Locate all <ul> elements

# Lists in Python: The result is a list, which can store multiple items.
print(sections[1])

<ul class="sub-menu">
<li class="menu-item menu-item-type-post_type menu-item-object-page menu-item-32" id="menu-item-32"><a href="https://starbucksmenuprices.com/starbucks-au-prices/">Australia</a></li>
<li class="menu-item menu-item-type-post_type menu-item-object-page menu-item-42" id="menu-item-42"><a href="https://starbucksmenuprices.com/starbucks-brasil-precos/">Brasil</a></li>
<li class="menu-item menu-item-type-post_type menu-item-object-page menu-item-70" id="menu-item-70"><a href="https://starbucksmenuprices.com/starbucks-%d1%86%d0%b5%d0%bd%d0%b8/">Bulgaria</a></li>
<li class="menu-item menu-item-type-post_type menu-item-object-page menu-item-58" id="menu-item-58"><a href="https://starbucksmenuprices.com/starbucks-canada-menu/">Canada</a></li>
</ul>


In [8]:
# Extract links from the <ul> sections
country_links = []  # Empty list to store results

for section in sections:
    links = section.find_all('a')  # Find all <a> tags in each section
    for link in links:
        country_name = link.text.strip()  # Get the visible text of the link
        country_url = link.get('href')  # Get the href attribute (URL)
        country_links.append({'Country': country_name, 'URL': country_url})

In [11]:
df = pd.DataFrame(country_links)

# Display the first few rows of the DataFrame
print(df.head())

     Country                                                URL
0        A-C                                                  #
1  Australia  https://starbucksmenuprices.com/starbucks-au-p...
2     Brasil  https://starbucksmenuprices.com/starbucks-bras...
3   Bulgaria  https://starbucksmenuprices.com/starbucks-%d1%...
4     Canada  https://starbucksmenuprices.com/starbucks-cana...


In [12]:
# Save the DataFrame to a CSV file
df.to_csv('starbucks_country_links.csv', index=False)
print("Saved country links to starbucks_country_links.csv")

Saved country links to starbucks_country_links.csv


In [1]:
links_file = 'starbucks_country_links.csv'  # File containing country links
country_links = pd.read_csv(links_file)  # Read the CSV file into a DataFrame

NameError: name 'pd' is not defined

In [ ]:
valid_links = country_links[~country_links['URL'].str.contains('#', na=False)]  # Filter rows without '#'
if valid_links.empty:
    print("No valid links found in the file.")
    exit()

first_link = valid_links.iloc[0]  # Select the first valid row
country_url = first_link['URL']  # Extract the URL from the first valid row
country_name = first_link['Country'].lower().replace(' ', '-')  # Extract and format the country name

In [ ]:
response = requests.get(country_url)
if response.status_code == 200:
    print(f"Successfully fetched the page: {country_url}")
else:
    print(f"Failed to fetch webpage. Status code: {response.status_code}")
    exit()

In [ ]:
soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
starbucks_prices_heading = soup.find('h2')  # Locate the "h2" section heading
if starbucks_prices_heading:
    starbucks_prices_table = starbucks_prices_heading.find_parent('table')  # Find the parent table containing the data
else:
    print("'Hot Coffee' section not found on the page.")
    exit()

In [ ]:
starbucks_prices_data = []  # List to store extracted data
if starbucks_prices_table:
    rows = starbucks_prices_table.find_all('tr', class_='item')  # Find all rows with class "item"
    for row in rows:
        cols = row.find_all('td')  # Find all columns in the row
        cols = [col.string.strip() for col in cols]  # Clean the text
        if cols:  # Skip empty rows
            starbucks_prices_data.append(cols)

In [ ]:
if starbucks_prices_data:
    df_starbucks_prices= pd.DataFrame(starbucks_prices_data, columns=['Item', 'Price'])  # Create a DataFrame
    output_file = f'starbucks_prices_{country_name}.csv'  # Generate file name with country name
    df_starbucks_prices.to_csv(output_file, index=False)  # Save to CSV
    print(f"Saved 'Starbucks Prices' prices to {output_file}")
    print(df_starbucks_prices)  # Print the result
else:
    print("No 'Starbucks Prices Data' data found.")